## Preparation

In [ ]:
!cat  /proc/cpuinfo

In [ ]:
!nvidia-smi -L

In [ ]:
%%capture
!pip install tensorly
!pip install --upgrade torch torchvision

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
print(torch.__version__)
torch.manual_seed(0)

from trainer import train, test
from tucker_layer import TuckerLayer

In [ ]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
train_data = torchvision.datasets.CIFAR10(root='./downloads/2013', train=True, download=True, transform=transform)
train_val_split = (int(len(train_data) * 0.8), len(train_data) - int(len(train_data) * 0.8))
train_data, val_data = torch.utils.data.random_split(train_data, train_val_split)
trainloader = torch.utils.data.DataLoader(train_data, batch_size=4, shuffle=True, num_workers=1)
valloader = torch.utils.data.DataLoader(val_data, batch_size=4, shuffle=False, num_workers=1)

test_data = torchvision.datasets.CIFAR10(root='./downloads/2013', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(test_data, batch_size=4, shuffle=False, num_workers=1)

In [ ]:
# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % labels[j] for j in range(4)))

## ALexNet

In [ ]:
AlexNet_model = torch.hub.load('pytorch/vision:v0.9.0', 'alexnet', pretrained=True)
AlexNet_model.eval()
AlexNet_model.classifier[1] = nn.Linear(9216,4096)
AlexNet_model.classifier[4] = nn.Linear(4096,1024)
AlexNet_model.classifier[6] = nn.Linear(1024,10)

In [ ]:
# AlexNet_model = AlexNet()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
AlexNet_model.to(device)
def loss_func1(model, output, labels):
    return nn.CrossEntropyLoss()(output, labels)
optimizer1 = optim.SGD(
    [
        {"params": AlexNet_model.features.parameters(), "lr": 0.0001, "momentum": 0.9},
        {"params": AlexNet_model.classifier.parameters(), "lr": 0.001, "momentum": 0.9},
    ],
    lr=0.0001,
)
scheduler1 = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer1, T_0=1, T_mult=2, verbose=False)

In [ ]:
train(model=AlexNet_model,
      train_loader=trainloader,
      val_loader=valloader,
      epoch_num=63,
      optimizer=optimizer1,
      scheduler=scheduler1,
      scheduler_step='batch',
      loss_func=loss_func1,
      path='alexnet')

In [ ]:
del(AlexNet_model)

## AlexNet Tucker Compression without orthogonal

In [ ]:
AlexTucker_model = torch.hub.load('pytorch/vision:v0.9.0', 'alexnet', pretrained=False)
AlexTucker_model.eval()
AlexTucker_model.classifier[1] = nn.Linear(9216,4096)
AlexTucker_model.classifier[4] = nn.Linear(4096,1024)
AlexTucker_model.classifier[6] = nn.Linear(1024,10)
AlexTucker_model.load_state_dict(torch.load('alexnet_best.pth'))
AlexTucker_model = AlexTucker_model.to('cpu')

In [ ]:
for i in [10]:
    AlexTucker_model.features[i] = TuckerLayer.from_Conv2D(AlexTucker_model.features[i], method='HOSVD')

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
AlexTucker_model.to(device)
optimizer2 = optim.SGD(
    [
        {"params": AlexTucker_model.features[10].parameters(), "lr": 0.0001, "momentum": 0.9},
    ],
    lr=0.00001, momentum=0.9
)
scheduler2 = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer2, T_0=1, T_mult=2, verbose=False)

In [ ]:
before_accu = test(AlexTucker_model, valloader)
print('Acuracy: {:.3f}'.format(before_accu * 100))

In [ ]:
train(model=AlexTucker_model,
      train_loader=trainloader,
      val_loader=valloader,
      epoch_num=63,
      optimizer=optimizer2,
      scheduler=scheduler2,
      scheduler_step='batch',
      loss_func=loss_func1,
      path='alexnet_tucker')

In [ ]:
del(AlexTucker_model)

## AlexNet Tucker Compression with orthogonal regularization

In [ ]:
AlexTucker_model = torch.hub.load('pytorch/vision:v0.9.0', 'alexnet', pretrained=False)
AlexTucker_model.eval()
AlexTucker_model.classifier[1] = nn.Linear(9216,4096)
AlexTucker_model.classifier[4] = nn.Linear(4096,1024)
AlexTucker_model.classifier[6] = nn.Linear(1024,10)
AlexTucker_model.load_state_dict(torch.load('alexnet_best.pth'))
AlexTucker_model = AlexTucker_model.to('cpu')

In [ ]:
for i in [10]:
    AlexTucker_model.features[i] = TuckerLayer.from_Conv2D(AlexTucker_model.features[i], method='HOSVD')

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
AlexTucker_model.to(device)
def loss_func2(model, output, labels):
    loss1 = nn.CrossEntropyLoss()(output, labels)
    loss2 = 0.0
    for i in [10]:
        loss2 += model.features[i].orthogonal_error()
    return loss1 + 0.01 * loss2
optimizer3 = optim.SGD(
    [
        {"params": AlexTucker_model.features[10].parameters(), "lr": 0.0001, "momentum": 0.9},
    ],
    lr=0.00001, momentum=0.9
)
scheduler3 = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer3, T_0=1, T_mult=2, verbose=False)

In [ ]:
train(model=AlexTucker_model,
      train_loader=trainloader,
      val_loader=valloader,
      epoch_num=63,
      optimizer=optimizer3,
      scheduler=scheduler3,
      scheduler_step='batch',
      loss_func=loss_func2,
      path='alexnet_tucker2')

In [ ]:
del(AlexTucker_model)

In [ ]:
def count_parameters(paras):
    return sum(p.numel() for p in paras if p.requires_grad)

def test_model_from_path(model_path):
    model = torch.jit.load(model_path + '.pt')
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    start_time = time.time()
    print(count_parameters(model.parameters()))
    print(count_parameters(model.features.parameters()))
    print('{:.3f}'.format(test(model, testloader) * 100))
    if torch.cuda.is_available():
        torch.cuda.synchronize()
    print(time.time() - start_time)

In [ ]:
test_model_from_path('alexnet')
test_model_from_path('alexnet_tucker')
test_model_from_path('alexnet_tucker2')
print(before_accu)